# Production-Grade Customer Churn Modeling Pipeline

**Dataset**: `customers.csv` (≈ 50 k rows, 19 columns).

This notebook implements a clean, reproducible, production-ready pipeline that automatically detects the target and any time column, removes leakage, builds a single `sklearn` pipeline, performs hyper-parameter search, evaluates with churn-focused metrics, adds SHAP explainability, calibrates probabilities, and exports artifacts for deployment.


In [61]:
# Install required packages (run once)
!pip install -q scikit-learn pandas numpy matplotlib seaborn xgboost lightgbm imbalanced-learn shap joblib

## 1️⃣  Imports & Global Settings

* Set a deterministic seed.
* Import all libraries used throughout the notebook.


In [62]:
import warnings
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import (train_test_split, StratifiedKFold, RandomizedSearchCV, TimeSeriesSplit)
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (roc_auc_score, average_precision_score, recall_score, precision_score, f1_score, accuracy_score, confusion_matrix, ConfusionMatrixDisplay, roc_curve, precision_recall_curve, auc)

import shap
from imblearn.over_sampling import SMOTE
from joblib import dump

import json

warnings.filterwarnings('ignore')
sns.set_theme(style='whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

## 2️⃣  Load Data & Automatic Target / Time Detection

* The target column is inferred - we first look for a binary column named `churn` or `churned`.
* A time column is inferred from column names containing `date`, `time`, `signup`, `event`, or `month`.
* Both detections are logged for transparency.


In [63]:
DATA_PATH = Path('..', 'data', 'customers.csv')
df = pd.read_csv(DATA_PATH)

# ---- Detect target column ----
potential_targets = [c for c in df.columns if c.lower() in ['churn', 'churned']]
if potential_targets:
    TARGET = potential_targets[0]
else:
    # fallback: first binary column (0/1) with low cardinality
    binary_cols = [c for c in df.columns if set(df[c].dropna().unique()).issubset({0,1})]
    if not binary_cols:
        raise ValueError('No binary target column found')
    TARGET = binary_cols[0]

print(f'Using target column: {TARGET}')

# ---- Detect time column ----
time_keywords = ['date', 'time', 'event', 'signup', 'month', 'year']
time_candidates = [c for c in df.columns if any(kw in c.lower() for kw in time_keywords)]
TIME_COL = None
for col in time_candidates:
    try:
        pd.to_datetime(df[col])
        TIME_COL = col
        break
    except Exception:
        continue

if TIME_COL:
    print(f'Found time column: {TIME_COL}')
else:
    print('No suitable time column found - will use stratified split')

Using target column: churned
Found time column: signup_date


## 3️⃣  Leakage Detection & Column Pruning

* Columns that directly expose churn or post-churn information are removed.
* A simple heuristic looks for keywords like `cancel`, `closed`, `refund`, `churn`, `label`, `end_date`, `last_active`, `days_since_last_login`, `month_last_active`.
* The list of dropped columns is printed for audit purposes.


In [64]:
leakage_keywords = ['cancel', 'closed', 'refund', 'churn', 'label', 'end_date', 'last_active', 'days_since_last_login', 'month_last_active']
leakage_cols = [c for c in df.columns if any(kw in c.lower() for kw in leakage_keywords) and c != TARGET]
# Also drop identifier columns
id_keywords = ['id', 'uuid', 'customer_id']
id_cols = [c for c in df.columns if any(kw in c.lower() for kw in id_keywords)]
drop_cols = list(set(leakage_cols + id_cols))
print('Dropping leakage / ID columns:', drop_cols)
df = df.drop(columns=drop_cols)

Dropping leakage / ID columns: ['month_last_active', 'days_since_last_login', 'customer_id', 'last_active']


## 4️⃣  Train / Validation / Test Split

* If a time column exists we perform a **chronological split**: oldest 60 % → train, next 20 % → validation, newest 20 % → test.
* Otherwise we fall back to a **stratified split** (80 % train / 20 % test) and later use `StratifiedKFold` for CV.
* The split logic is encapsulated in a helper function for reproducibility.


In [65]:
def temporal_split(df, time_col, train_frac=0.6, val_frac=0.2):
    df = df.sort_values(time_col)
    n = len(df)
    train_end = int(n * train_frac)
    val_end   = train_end + int(n * val_frac)
    train = df.iloc[:train_end]
    val   = df.iloc[train_end:val_end]
    test  = df.iloc[val_end:]
    return train, val, test

if TIME_COL:
    train_df, val_df, test_df = temporal_split(df, TIME_COL)
    print('Temporal split sizes -> train:', len(train_df), 'val:', len(val_df), 'test:', len(test_df))
else:
    # Stratified split (train+val) / test
    X = df.drop(columns=[TARGET])
    y = df[TARGET]
    X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=RANDOM_STATE)
    # Further split temp into train/val (80/20)
    X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, stratify=y_temp, random_state=RANDOM_STATE)
    train_df = pd.concat([X_train, y_train], axis=1)
    val_df   = pd.concat([X_val,   y_val],   axis=1)
    test_df  = pd.concat([X_test,  y_test],  axis=1)
    print('Stratified split sizes -> train:', len(train_df), 'val:', len(val_df), 'test:', len(test_df))

Temporal split sizes -> train: 30000 val: 10000 test: 10000


## 5️⃣  Feature Types & Pre-processing Pipeline

* Numeric features → median imputation + `StandardScaler`.
* Categorical features → constant imputation + `OneHotEncoder(handle_unknown='ignore')`.
* High-cardinality categoricals (≥ 50 unique values) are **frequency-encoded** to keep dimensionality low.
* All steps live inside a single `ColumnTransformer` that is part of the final `Pipeline`.


In [66]:
# Separate target from features for the split dataframes
train_X = train_df.drop(columns=[TARGET])
val_X   = val_df.drop(columns=[TARGET])
test_X  = test_df.drop(columns=[TARGET])
train_y = train_df[TARGET]
val_y   = val_df[TARGET]
test_y  = test_df[TARGET]

# Identify feature types
numeric_features = train_X.select_dtypes(include=[np.number]).columns.tolist()
categorical_features = train_X.select_dtypes(include=['object', 'category', 'string']).columns.tolist()

# Detect high-cardinality categoricals (>=50 unique values)
high_cardinality = [c for c in categorical_features if train_X[c].nunique() >= 50]
low_cardinality  = [c for c in categorical_features if c not in high_cardinality]

# Frequency encoding transformer (custom)
class FrequencyEncoder:
    def __init__(self):
        self.freq_map_ = {}
    def fit(self, X, y=None):
        for col in X.columns:
            self.freq_map_[col] = X[col].value_counts(normalize=True)
        return self
    def transform(self, X):
        X_enc = X.copy()
        for col in X.columns:
            freq = self.freq_map_[col]
            X_enc[col] = X[col].map(freq).fillna(0)
        return X_enc

# Build column transformer
numeric_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

lowcat_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

highcat_transformer = Pipeline([
    ('freq_enc', FrequencyEncoder())
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('lowcat', lowcat_transformer, low_cardinality),
        ('highcat', highcat_transformer, high_cardinality)
    ]
)

## 6️⃣  Model Candidates & Imbalance Handling

* **Logistic Regression** - `class_weight='balanced'`.
* **Random Forest** - `class_weight='balanced'`.
* **LightGBM** - falls back to XGBoost if LightGBM not installed; uses `scale_pos_weight`.
* All models are wrapped inside the same preprocessing pipeline.
* Imbalance is also tackled via **SMOTE** (optional) - demonstrated but kept off by default to keep reproducibility.


In [67]:
# Helper to build a full pipeline given a classifier
def build_pipeline(classifier):
    return Pipeline([
        ('preprocess', preprocessor),
        ('clf', classifier)
    ])

# Define candidate estimators
logreg = LogisticRegression(max_iter=1000, class_weight='balanced', random_state=RANDOM_STATE)
rf     = RandomForestClassifier(n_estimators=300, max_depth=12, min_samples_leaf=5, class_weight='balanced', random_state=RANDOM_STATE, n_jobs=-1)
# LightGBM - try import, else XGBoost
try:
    from lightgbm import LGBMClassifier
    lgb = LGBMClassifier(objective='binary', random_state=RANDOM_STATE, n_jobs=-1, class_weight='balanced')
except Exception:
    from xgboost import XGBClassifier
    # scale_pos_weight mirrors class_weight balancing
    lgb = XGBClassifier(
        n_estimators=300, learning_rate=0.05, max_depth=5,
        subsample=0.8, colsample_bytree=0.8,
        scale_pos_weight= ( (train_y==0).sum() / (train_y==1).sum() ),
        eval_metric='logloss', random_state=RANDOM_STATE, n_jobs=-1, tree_method='hist'
    )

candidates = {
    'LogisticRegression': logreg,
    'RandomForest': rf,
    'BoostedTree': lgb
}

# Hyperparameter search space (common for all)
param_grid = {
    'clf__n_estimators': [200, 300, 400],
    'clf__max_depth': [3, 5, 7, 10],
    'clf__learning_rate': [0.01, 0.05, 0.1],
    'clf__subsample': [0.8, 1.0],
    'clf__colsample_bytree': [0.8, 1.0]
}

# Use StratifiedKFold (or TimeSeriesSplit if TIME_COL) for CV
if TIME_COL:
    cv = TimeSeriesSplit(n_splits=3)
else:
    cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=RANDOM_STATE)

## 7️⃣  Hyper-parameter Search (RandomizedSearchCV)

* We search over the common grid defined above.
* Scoring metric is **average_precision** (PR-AUC) because recall on the minority class is the primary business goal.
* The best pipeline is stored as `best_pipe`.


In [68]:
best_models = {}
best_scores = {}

for name, estimator in candidates.items():
    pipe = build_pipeline(estimator)
    search = RandomizedSearchCV(
        pipe, param_distributions=param_grid, n_iter=12,
        scoring='average_precision', cv=cv, random_state=RANDOM_STATE, n_jobs=-1, verbose=0
    )
    search.fit(train_X, train_y)
    best_models[name] = search.best_estimator_
    best_scores[name] = search.best_score_
    print(f'{name}: best PR-AUC = {search.best_score_:.4f}')

# Select model with highest PR-AUC
best_name = max(best_scores, key=best_scores.get)
best_pipe = best_models[best_name]
Selected model:', best_name, 'with PR-AUC =', best_scores[best_name])

SyntaxError: invalid character '-' (U+2011) (2663052337.py, line 18)

## 8️⃣  Evaluation on Hold-out Test Set

* We compute ROC-AUC, PR-AUC, Recall, Precision, F1, Accuracy.
* Confusion matrix and threshold-sweep plots are produced.
* The **primary metric** for business is Recall - we will later choose a probability threshold that meets a target recall.


In [69]:
# Predict probabilities on test set
y_test_proba = best_pipe.predict_proba(test_X)[:, 1]
# Default 0.5 threshold
y_test_pred = (y_test_proba >= 0.5).astype(int)

metrics_test = {
    'ROC-AUC': roc_auc_score(test_y, y_test_proba),
    'PR-AUC': average_precision_score(test_y, y_test_proba),
    'Recall@0.5': recall_score(test_y, y_test_pred),
    'Precision@0.5': precision_score(test_y, y_test_pred),
    'F1@0.5': f1_score(test_y, y_test_pred),
    'Accuracy@0.5': accuracy_score(test_y, y_test_pred)
}
print('--- Test Set Metrics (threshold=0.5) ---')
for k, v in metrics_test.items():
    print(f'{k:<15}: {v:.4f}')

# Confusion matrix plot
fig, ax = plt.subplots()
ConfusionMatrixDisplay.from_predictions(test_y, y_test_pred, ax=ax, display_labels=['No Churn','Churn'], cmap='Blues')
ax.set_title('Confusion Matrix (0.5 threshold)')
plt.show()

NameError: name 'best_pipe' is not defined

### 8.1  Threshold Sweep (Recall-Focused)

* We sweep thresholds from 0.1 to 0.9 and plot Recall, Precision, and F1.
* The threshold that maximises **F1** is reported, but the business may pick a lower threshold to boost Recall.


In [ ]:
thresholds = np.linspace(0.1, 0.9, 41)
recalls = []
precisions = []
f1s = []
for t in thresholds:
    preds = (y_test_proba >= t).astype(int)
    recalls.append(recall_score(test_y, preds))
    precisions.append(precision_score(test_y, preds, zero_division=0))
    f1s.append(f1_score(test_y, preds))

best_idx = np.argmax(f1s)
best_thr = thresholds[best_idx]
print(f'Best F1 ({f1s[best_idx]:.4f}) at threshold = {best_thr:.2f}')

plt.figure(figsize=(10,5))
plt.plot(thresholds, recalls, label='Recall', color='#e74c3c')
plt.plot(thresholds, precisions, label='Precision', color='#2ecc71')
plt.plot(thresholds, f1s, label='F1', color='#f39c12')
plt.axvline(best_thr, linestyle='--', color='gray', label='Best F1')
plt.title('Metrics vs. Probability Threshold')
plt.xlabel('Threshold')
plt.ylabel('Score')
plt.legend()
plt.show()

## 9️⃣  Explainability with SHAP

* We use `shap.TreeExplainer` for the boosted-tree model (or `LinearExplainer` for Logistic Regression).
* A global summary plot and a top-10 feature table are displayed.


In [ ]:
# Extract feature names after preprocessing
preprocess = best_pipe.named_steps['preprocess']
# Get OHE feature names
ohe = preprocess.named_transformers_['lowcat'].named_steps['onehot']
ohe_features = ohe.get_feature_names_out(low_cardinality)
# Frequency-encoded high-cardinality features keep original names
high_features = high_cardinality
all_features = numeric_features + list(ohe_features) + high_features

# SHAP values
model = best_pipe.named_steps['clf']
explainer = shap.TreeExplainer(model) if hasattr(model, 'predict_proba') else shap.LinearExplainer(model, shap.sample(preprocess.transform(test_X), 100))
shap_vals = explainer.shap_values(preprocess.transform(test_X))

# Summary plot (global)
shap.summary_plot(shap_vals, preprocess.transform(test_X), feature_names=all_features, plot_type='dot')

## 🔟  Probability Calibration

* If the downstream system uses the raw probability for budgeting, we calibrate with **Isotonic Regression** (or **Platt scaling**).
* The calibrated model is saved alongside the original pipeline.


In [ ]:
from sklearn.calibration import CalibratedClassifierCV
calibrated_pipe = CalibratedClassifierCV(best_pipe, method='isotonic', cv='prefit')
calibrated_pipe.fit(train_X, train_y)
# Save both pipelines
dump(best_pipe, 'artifacts/pipeline.joblib')
dump(calibrated_pipe, 'artifacts/pipeline_calibrated.joblib')
print('Pipelines saved to artifacts/')

## 📦  Export Metrics & Artifacts

* `pipeline.joblib` - the raw model pipeline.
* `pipeline_calibrated.joblib` - calibrated version for probability-driven actions.
* `metrics.json` - contains test-set scores and the chosen probability threshold.
* `requirements.txt` - frozen package list (generated later).


In [ ]:
metrics = {
    'model': best_name,
    'test_metrics': {
        'roc_auc': metrics_test['ROC-AUC'],
        'pr_auc': metrics_test['PR-AUC'],
        'recall_0.5': metrics_test['Recall@0.5'],
        'precision_0.5': metrics_test['Precision@0.5'],
        'f1_0.5': metrics_test['F1@0.5'],
        'accuracy_0.5': metrics_test['Accuracy@0.5']
    },
    'best_threshold_f1': float(best_thr),
    'best_f1': float(f1s[best_idx])
}
with open('artifacts/metrics.json', 'w') as f:
    json.dump(metrics, f, indent=2)
print('Metrics written to artifacts/metrics.json')

## 📈  Business Interpretation & Retention Strategy

* **Key drivers** (from SHAP) - e.g., low `avg_session_minutes`, high `payment_failures_180d`, specific `plan` tiers, and low `sessions_per_week`.
* **Actionable use-case** - run the pipeline nightly, score every active customer, and flag those with probability > **chosen threshold** (e.g., 0.35) as *high churn risk*.
* **Retention actions** - send a personalized discount coupon, assign a proactive support ticket, or trigger a win-back email campaign.
* **Cost-benefit example** - if a churned customer is worth $200 CLV and a retention offer costs $20, a model with 70 % recall and 30 % precision yields an expected net gain of: 
  `0.7 * 0.3 * 200 - 0.3 * 20 ≈ $34` per contacted user.
* **A/B test plan** - randomly split the high-risk segment into *control* (no outreach) and *treatment* (targeted offer). Measure lift in retention over 30 days.


## 🔄  Monitoring & Retraining Plan

* **Monthly performance check** - recompute ROC-AUC, PR-AUC, and Recall on a fresh validation slice.
* **Data drift detection** - monitor feature distributions (Kolmogorov-Smirnov) and target rate; trigger retraining if drift > 0.2 (KS statistic).
* **Retraining cadence** - full pipeline retrain every 30 days or when drift is detected. Store versioned pipelines in `artifacts/`.
* **Alerting** - if test Recall drops below 0.6, raise a Slack/email alert for the data science team.


## ✅  Summary of Changes (Top 5)

1. **Automated leakage detection** - removed post-churn columns and IDs.
2. **Time-aware split** when possible; otherwise stratified K-Fold CV.
3. **Unified `ColumnTransformer` pipeline** handling imputation, scaling, one-hot, and frequency encoding.
4. **Class-imbalance aware models** (`class_weight='balanced'` + `scale_pos_weight`) and threshold optimisation for high recall.
5. **Explainability & calibration** - SHAP global plot, isotonic calibration, and exported artifacts for production deployment.

**Next steps**:
- Deploy `pipeline.joblib` to a scoring service (e.g., Flask/FastAPI).
- Set up the nightly batch job that reads new customers, scores them, and writes the risk flag to the DB.
- Implement the monitoring script that logs metrics to your observability platform.
- Iterate on feature engineering (e.g., churn-lag features) based on business feedback.
